In [2]:
import csv
import json
import os
import re

In [3]:
# os.listdir("../../data")
datadir = "../../data/"

In [4]:
def get_associated_twitters(text):
    """
    Parses a block of text and returns a list of twitter accounts mentioned in the text.
    """
    
    accounts = list(set(re.findall("(?<=twitter.com\/)\w+(?=\/)", text)))
    return accounts

In [5]:
# Read all files and store processed dictionary in list
processed_data = []
# for file in os.listdir(datadir)[42760:42768]:
for i, file in enumerate(os.listdir(datadir)):
    with open(datadir + file, 'r') as f:
    # read file
        try:
            d = json.loads(json.load(f))
        except:
            pass
        
        mirrorDict = dict()
        # store data
        try:
            arweaveTx = file.split(".json")[0]

            mirrorDict["contributor"] = d["authorship"]["contributor"];
            mirrorDict["title"] = d["content"]["title"]
            mirrorDict["date_published"] = d["content"]["timestamp"]
            mirrorDict["body"] = d["content"]["body"]
            try:
                mirrorDict["digest"] = d["originalDigest"]
            except:
                mirrorDict["digest"] = d["digest"]
            try:
                mirrorDict["publication"] = d["publication"]
            except:
                mirrorDict["publication"] = ""
            mirrorDict
            mirrorDict["associated_twitters"] = get_associated_twitters(mirrorDict["body"])
            mirrorDict["arweave_tx"] = arweaveTx

            processed_data.append(mirrorDict)
        except:
            print(arweaveTx, d)
        
f.close()

## Articles table

Each row becomes a node with Article as the label (and Mirror) as a label

| contributor | digest | date_published | title | arweave_tx |
|--|--|--|--|--|
| 0xa39C9Ce81c6FaC173578658Ac8FEbf60eb5a3513 | prMJHUw4m_1E_KM5Yyo7uOUEtVX-tJR3_kzs4zKhwZ4 | 1642280045 | NFTs Are Critical for the Future of Art - OneZero | dafadsfa 

## Wallets

Each row becomes a node with Wallet as the label

| wallet |
|--|
| 0xa39C9Ce81c6FaC173578658Ac8FEbf60eb5a3513 

## Accounts

Each row becomes a node with Account as a label (and Twitter as a label)

| username | digest |
|--|--|
| 

In [23]:
# [(i, d["associated_twitters"]) for i,d in enumerate(processed_data)]
[d for d in processed_data if d["digest"] == "CAA3e6zD81jfofKRyYV4briD6Q3FRZESKmGKFW8jJ8U"][0]["title"].replace('\\', '')

'hello  ,guys~~\\'

## Create csv files for Neo4j ingest

In [10]:
processed_data[24900]

{'contributor': '0x248569ed129d725E9C99858995Cf8Ae820C514e3',
 'title': 'linux 查看自己所在的公网ip',
 'date_published': 1635305988,
 'body': 'curl\xa0members.3322.org/dyndns/getip\n\n其他的方法还有:\n\n```\ncurl icanhazip.com  \ncurl ifconfig.me  \ncurl curlmyip.com  \ncurl ip.appspot.com  \ncurl ipinfo.io/ip  \ncurl ipecho.net/plain  \ncurl www.trackip.net/i  \n```',
 'digest': 'Yb5BdlXlnPZqpzt_tTIP9E6qp-zMQZUAlaVSWIbi4qA',
 'publication': '',
 'associated_twitters': [],
 'arweave_tx': 'oYCVj5VHaqJcC0qzTT1bwEBBq8s2QbMODpXiNbcheg8'}

### Create Articles table

In [24]:
# contributor	digest	date_published	title
articles = [
    {
        "contributor": r["contributor"],
        "digest": r["digest"],
        "arweave_tx": r["arweave_tx"],
        "date_published": r["date_published"],
        "title": r["title"].replace('\\', '') # need to escape '\' for neo4j formatting
    } for r in processed_data
]

keys = articles[0].keys()
# print(articles)
with open("./data/neo4j-ingest/articles.csv", "w", encoding="utf-8") as f:
    fcsv = csv.DictWriter(f, keys)
    fcsv.writeheader()
    fcsv.writerows(articles)
f.close()

In [177]:
processed_data[3]

{'contributor': '0xc767778E9B7841C029966f05B9762d3342e0510A',
 'title': 'CGV Research: Will Treasure, Run by DAOs, Be the “Nintendo” in the Web3 World?',
 'date_published': 1642581301,
 'body': '\\*— \\*By: Vargason, CGV FoF Research Fellow\n\nThe continuous expansion of metaverse projects exacerbates homogenization. The strategy of betting on projects that can continuously expand the ecosystem, empower games and NFTs may be better than betting on new projects.\n\n![](https://miro.medium.com/max/1400/1\\*TxxSt11X-nMVXl2-5BlLTQ.jpeg)\n\nWhile most industries have struggled to survive since the outbreak of the Covid-19 pandemic in 2020, the Axie Infinity of GameFi has swept the globe, creating wealth for many families and attracting a host of users outside the cryptocurrency world to participate in the blockchain game featured with “Play to Earn”.\n\nAt a time when more and more projects focus only on making games, CGV finds that the Treasure run by DAOs that blurs the boundaries between

### Create Wallets table

In [26]:
wallets = [
    {
        "contributor": r["contributor"]
    } for r in processed_data
]

# keys = wallets[0].keys()
# # print(articles)
# with open("./data/neo4j-ingest/wallets.csv", "w") as f:
#     fcsv = csv.DictWriter(f, keys)
#     fcsv.writeheader()
#     fcsv.writerows(wallets)
# f.close()

In [34]:
len(list(set([d["contributor"] for d in wallets])))

21821

### Create Accounts table

In [162]:
# processed_data[28]

In [174]:
accounts = [
    {
    "digest": d["digest"],
    "linked_twitter": s
    }
    for d in processed_data for s in d["associated_twitters"]
]

keys = accounts[0].keys()
# print(articles)
with open("./data/neo4j-ingest/accounts.csv", "w") as f:
    fcsv = csv.DictWriter(f, keys)
    fcsv.writeheader()
    fcsv.writerows(accounts)
f.close()

# Neo4j Load Scripts

In [ ]:
# Articles nodes
#         "contributor": r["contributor"],
#         "digest": r["digest"],
#         "arweave_tx": r["arweave_tx"],
#         "date_published": r["date_published"],
#         "title": r["title"]
LOAD CSV WITH HEADERS FROM 'https://chainverse.s3.us-east-2.amazonaws.com/neo/mirror/nodes/articles.csv' AS row
MERGE (article:Article:Mirror {digest: row.digest})
  ON CREATE SET article.contributor = row.contributor, article.arweave_tx = row.arweave_tx, article.date_published = row.date_published, article.title = row.title;

In [ ]:
# Wallet nodes
# wallets = [
#     {
#         "contributor": r["contributor"]
#     } for r in processed_data[0:400]
# ]
LOAD CSV WITH HEADERS FROM 'https://chainverse.s3.us-east-2.amazonaws.com/neo/mirror/nodes/wallets.csv' AS row
MERGE (wallet:Wallet {address: row.contributor});

In [ ]:
# Account nodes
# accounts = [
#     {
#     "digest": d["digest"],
#     "linked_twitter": s
#     }
#     for d in processed_data[0:400] for s in d["associated_twitters"]
# ]
LOAD CSV WITH HEADERS FROM 'https://chainverse.s3.us-east-2.amazonaws.com/neo/mirror/nodes/accounts.csv' AS row
MERGE (account:Account:Twitter {username: row.linked_twitter})

## Create indexes

In [ ]:
CREATE INDEX digest FOR (a:Article) ON (a.digest);
CREATE INDEX wallet_address FOR (w:Wallet) ON (w.address);
CREATE CONSTRAINT digest ON (a:Article) ASSERT a.address IS UNIQUE;
CREATE CONSTRAINT wallet_address ON (w:Wallet) ASSERT w.address IS UNIQUE;
CALL db.awaitIndexes();

### Create relations

In [ ]:
# Wallet to Article
LOAD CSV WITH HEADERS FROM 'https://chainverse.s3.us-east-2.amazonaws.com/neo/mirror/nodes/articles.csv' AS row
MATCH (article:Article {digest: row.digest})
MATCH (wallet:Wallet {address: row.contributor})
MERGE (wallet)-[wc:PUBLISHED]->(article);
#   ON CREATE SET op.unitPrice = toFloat(row.UnitPrice), op.quantity = toFloat(row.Quantity);

In [ ]:
# Article to Twitter account
LOAD CSV WITH HEADERS FROM 'https://chainverse.s3.us-east-2.amazonaws.com/neo/mirror/nodes/accounts.csv' AS row
MATCH (account:Account {username: row.linked_twitter})
MATCH (article:Article {digest: row.digest})
MERGE (article)-[aa:REFERENCES]->(account);
